In [159]:
%run runaway_functionsv3
cl = Cluster("Berkeley_97")
# cl.mymembers

In [160]:
cld = ClusterDias("Berkeley_97")
cld.members()

RAdeg,DEdeg,Source,Pmemb,Plx,e_Plx,pmRA,e_pmRA,pmDE,e_pmDE,Gmag,e_Gmag,BP-RP,e_BP-RP
deg,deg,,,mas,mas,mas / yr,mas / yr,mas / yr,mas / yr,mag,mag,mag,mag
float64,float64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
339.92506216487,58.96974621703,2008383646002225536,0.4,0.271,0.076,-3.158,0.119,-1.868,0.112,17.0656,0.0008,1.37,0.020499999999999997
339.87673134165,58.96071295168,2008382202893209600,0.6,0.3461,0.0364,-2.823,0.056,-2.151,0.058,15.8737,0.0005,1.2833,0.008199999999999999
339.94579695616,59.02046081911,2008384642434841984,0.4,0.1572,0.0881,-2.536,0.148,-1.724,0.143,17.2479,0.001,2.729,0.0327
340.10268404895,58.97390331535,2008380794144211712,0.9,0.2935,0.0271,-2.72,0.043,-1.857,0.048,11.5058,0.0019,1.1894,0.01
339.99717508578,58.92056823106,2008380240083932160,0.5,0.4375,0.1033,-2.556,0.157,-1.913,0.159,17.8065,0.0012,1.6559,0.020999999999999998
340.07597705392,58.95726378513,2008380691065000448,0.4,0.2248,0.113,-2.534,0.174,-1.597,0.19,17.7075,0.0011,1.5773,0.0323
340.07972172779,58.94999243161,2008380622345529216,0.5,0.1883,0.0283,-2.704,0.046,-1.773,0.048,14.2904,0.0003,2.4564,0.0056
339.8456407694,59.00830729229,2008383302396657664,0.8,0.2218,0.0629,-2.419,0.103,-1.707,0.084,15.9679,0.0036,1.107,0.1679
